In [1]:
import numpy as np
import cv2
import os
os.chdir('./data/')

<h1>Create checkerboard imge</h1>

In [2]:
def create_checkerboard(file_name, square_size_cm):
    rows, cols = 8, 8
    px_per_cm = 100 # 100 pixel for cm

    square_size_px = int(square_size_cm * px_per_cm)
    checkerboard = np.zeros((rows * square_size_px, cols * square_size_px), dtype=np.uint8)

    for i in range(rows):
        for j in range(cols):
            if (i + j) % 2 == 0:
                checkerboard[i * square_size_px:(i + 1) * square_size_px, j * square_size_px:(j + 1) * square_size_px] = 255
                cv2.imshow("Checkerboard", checkerboard)
                cv2.waitKey(0)
    
    cv2.destroyAllWindows()            
    cv2.imwrite(file_name, checkerboard)

In [3]:
create_checkerboard("checkerboard.png", 2)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ahmedcr7/.local/lib/python3.11/site-packages/cv2/qt/plugins"


<h1>Collect images for calibration</h1>

In [67]:
# Corners of the checkerboard
CHESS_BOARD_DIMENSIONS = (7, 7)
num_of_saved_images = 0
SQUARE_SIZE = 25 # mm

image_path = "chess_board_images/"

check_dir = os.path.isdir(image_path)
if check_dir:
    print("directory already exists")
else:
    os.mkdir(image_path)
    print("directory created")

directory already exists


In [23]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [24]:
def detect_checker_board(image,gray_image,criteria,board_dim):
    ret, corners = cv2.findChessboardCorners(gray_image, board_dim)
    if ret:
        improve_corners = cv2.cornerSubPix(gray_image,corners,(3,3),(-1,-1),criteria) # gray_image, corners, window_size, zero_zone, criteria
        image = cv2.drawChessboardCorners(image, board_dim, improve_corners, ret)
    
    return image,ret

In [25]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame,(940,880))
    copy_frame = frame.copy()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    image, board_detection = detect_checker_board(frame,gray,criteria,CHESS_BOARD_DIMENSIONS)

    cv2.putText(frame,f"saved_images : {num_of_saved_images}",(30,40),cv2.FONT_HERSHEY_COMPLEX,1.4,(0,255,0),2,cv2.LINE_AA)

    cv2.imshow('frame',frame)
    cv2.imshow('copy_frame',copy_frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
    
    if key == ord('s') and board_detection:
        num_of_saved_images += 1
        cv2.imwrite(f"./chess_board_images/image_{num_of_saved_images}.png",copy_frame)

cap.release()
cv2.destroyAllWindows()

<h1>extract intrinsic and extrinsic parameters using checkerboard</h1>

In [66]:
# # how get the chessboard corners

# # 49 num of corners   3 is x,y,z
# obj = np.zeros((7*7,3), np.float32)
# mesh_grid = np.mgrid[0:7,0:7].T.reshape(-1,2)# shape (49,2)
# obj[:,:2] = mesh_grid

# # multiply every coordinate by 20 mm to get the real world coordinates
# obj *= 20
# obj

array([[  0.,   0.,   0.],
       [ 20.,   0.,   0.],
       [ 40.,   0.,   0.],
       [ 60.,   0.,   0.],
       [ 80.,   0.,   0.],
       [100.,   0.,   0.],
       [120.,   0.,   0.],
       [  0.,  20.,   0.],
       [ 20.,  20.,   0.],
       [ 40.,  20.,   0.],
       [ 60.,  20.,   0.],
       [ 80.,  20.,   0.],
       [100.,  20.,   0.],
       [120.,  20.,   0.],
       [  0.,  40.,   0.],
       [ 20.,  40.,   0.],
       [ 40.,  40.,   0.],
       [ 60.,  40.,   0.],
       [ 80.,  40.,   0.],
       [100.,  40.,   0.],
       [120.,  40.,   0.],
       [  0.,  60.,   0.],
       [ 20.,  60.,   0.],
       [ 40.,  60.,   0.],
       [ 60.,  60.,   0.],
       [ 80.,  60.,   0.],
       [100.,  60.,   0.],
       [120.,  60.,   0.],
       [  0.,  80.,   0.],
       [ 20.,  80.,   0.],
       [ 40.,  80.,   0.],
       [ 60.,  80.,   0.],
       [ 80.,  80.,   0.],
       [100.,  80.,   0.],
       [120.,  80.,   0.],
       [  0., 100.,   0.],
       [ 20., 100.,   0.],
 

In [68]:
calibration_data_path = 'calibration_data'

check_dir = os.path.isdir(calibration_data_path)

if check_dir:
    print("directory already exists")

else:
    os.mkdir(calibration_data_path)
    print("directory created")

directory created


In [72]:
obj_3D = np.zeros((CHESS_BOARD_DIMENSIONS[0]*CHESS_BOARD_DIMENSIONS[1],3), np.float32) # 7*7*3
# get real world coordinates of chess board corners
obj_3D[:,:2] = np.mgrid[0:CHESS_BOARD_DIMENSIONS[0],0:CHESS_BOARD_DIMENSIONS[1]].T.reshape(-1,2)
# real distance between corners
obj_3D *= SQUARE_SIZE
print(obj_3D)

obj_3D_points = [] # 3D points in real world space
img_2D_points = [] # 2D points in image plane


images = os.listdir(image_path)
for Image in images:
    PATH = os.path.join(image_path,Image)

    img = cv2.imread(PATH)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHESS_BOARD_DIMENSIONS, None)
    if ret == True:
        obj_3D_points.append(obj_3D)
        improve_corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        img_2D_points.append(improve_corners)

        img = cv2.drawChessboardCorners(img,CHESS_BOARD_DIMENSIONS,improve_corners,ret)
    cv2.imshow('img',img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

[[  0.   0.   0.]
 [ 25.   0.   0.]
 [ 50.   0.   0.]
 [ 75.   0.   0.]
 [100.   0.   0.]
 [125.   0.   0.]
 [150.   0.   0.]
 [  0.  25.   0.]
 [ 25.  25.   0.]
 [ 50.  25.   0.]
 [ 75.  25.   0.]
 [100.  25.   0.]
 [125.  25.   0.]
 [150.  25.   0.]
 [  0.  50.   0.]
 [ 25.  50.   0.]
 [ 50.  50.   0.]
 [ 75.  50.   0.]
 [100.  50.   0.]
 [125.  50.   0.]
 [150.  50.   0.]
 [  0.  75.   0.]
 [ 25.  75.   0.]
 [ 50.  75.   0.]
 [ 75.  75.   0.]
 [100.  75.   0.]
 [125.  75.   0.]
 [150.  75.   0.]
 [  0. 100.   0.]
 [ 25. 100.   0.]
 [ 50. 100.   0.]
 [ 75. 100.   0.]
 [100. 100.   0.]
 [125. 100.   0.]
 [150. 100.   0.]
 [  0. 125.   0.]
 [ 25. 125.   0.]
 [ 50. 125.   0.]
 [ 75. 125.   0.]
 [100. 125.   0.]
 [125. 125.   0.]
 [150. 125.   0.]
 [  0. 150.   0.]
 [ 25. 150.   0.]
 [ 50. 150.   0.]
 [ 75. 150.   0.]
 [100. 150.   0.]
 [125. 150.   0.]
 [150. 150.   0.]]


In [74]:
RError, mtx, dist, RVecs, TVecs = cv2.calibrateCamera(obj_3D_points, img_2D_points, gray.shape[::-1], None, None) # real world points, image points, image shape, camera matrix, distortion coefficients
np.savez(
    f"{calibration_data_path}/MultiMatrix",
    cam_matrix=mtx,
    dist_coeffs=dist,
    rvecs=RVecs,
    tvecs=TVecs)

<h3>Load the Multi_Matrix</h3>

In [76]:
data = np.load('./calibration_data/MultiMatrix.npz')
cam_matrix = data['cam_matrix']
dist_matrix = data['dist_coeffs']
rvecs = data['rvecs']
tvecs = data['tvecs']

print(cam_matrix)
print(dist_matrix)
print(rvecs)
print(tvecs)

[[731.12850269   0.         410.04394907]
 [  0.         904.13035128 446.38632799]
 [  0.           0.           1.        ]]
[[-0.00981295 -0.74078757 -0.00354336 -0.01916192  0.60649542]]
[[[-0.09547118]
  [ 0.19873483]
  [ 1.46920487]]

 [[ 0.28075432]
  [ 1.20660571]
  [ 1.42607179]]

 [[-0.46151178]
  [ 1.2663048 ]
  [ 1.3980455 ]]

 [[ 0.2118138 ]
  [ 0.61805336]
  [ 1.35006007]]

 [[-0.08414572]
  [ 0.04420779]
  [ 0.90402126]]

 [[-0.2568581 ]
  [-0.61709611]
  [ 0.22028387]]

 [[-0.5896251 ]
  [ 0.1404878 ]
  [ 0.29767216]]

 [[ 0.39386409]
  [ 0.28481143]
  [-0.06939852]]

 [[-0.04042072]
  [ 0.3093176 ]
  [ 1.40182229]]

 [[-0.17542522]
  [ 0.51575994]
  [ 1.51628604]]]
[[[  64.66486459]
  [-102.6416511 ]
  [ 232.34412613]]

 [[ 223.96515359]
  [ -11.38079323]
  [ 312.19750661]]

 [[ 248.10753331]
  [  -9.53508693]
  [ 436.80479998]]

 [[ -44.63421643]
  [  -9.2439647 ]
  [ 568.06970546]]

 [[ -11.81756783]
  [ -82.67822216]
  [ 281.35362499]]

 [[-149.12262149]
  [ -55.326